In [78]:
# To start by reading the data frame of the samples of interest that contains the TPM expression 
# for protein coding genes only

# Although GTEX provides also the RAW COUNTS, given the large number of samples, 
# very likely, it is computationally prohibitve to compute the differential expresson with edgeR, DESEq2, or limma/voom 
# between liver tissue and all other tissues

setwd("/Users/tanasab/Desktop/GTEX")

In [4]:
suppressMessages({

library(data.table, quietly = TRUE)
library(plyr, quietly = TRUE)
library(doParallel,quietly = TRUE)
library(tidyverse, quietly = TRUE)
library("stringr", quietly = TRUE)

})

cores <- detectCores()
cores
registerDoParallel(cores=cores)
options(future.globals.maxSize = Inf)

[1] 10

In [5]:
a = read.table("df4.pc.samples.of.interest.txt", header=T, sep="\t", stringsAsFactors=F)

In [6]:
name = basename("df4.pc.samples.of.interest.txt")
print(name)

[1] "df4.pc.samples.of.interest.txt"


In [123]:
dim(a)
head(a,1)

length(rownames(a))
length(colnames(a))

[1] 18446  4421

,GTEX.1117F.0726.SM.5GIEN,GTEX.111FC.0626.SM.5N9CU,GTEX.111FC.0826.SM.5GZWO,GTEX.111VG.0326.SM.5GZX7,GTEX.111YS.0326.SM.5GZZ3,GTEX.111YS.0426.SM.5987O,GTEX.1122O.0626.SM.5N9B9,GTEX.1122O.0826.SM.5GICV,GTEX.117XS.0726.SM.5H131,GTEX.117YW.0326.SM.5N9CY,⋯,GTEX.ZY6K.0226.SM.5SIAY,GTEX.ZYFC.0926.SM.5GZWW,GTEX.ZYFD.0826.SM.5NQ9A,GTEX.ZYFG.0626.SM.5GZYA,GTEX.ZYT6.0426.SM.5GID3,GTEX.ZYVF.1126.SM.5E458,GTEX.ZYW4.1126.SM.5SI99,GTEX.ZYY3.1926.SM.5GZXS,GTEX.ZZ64.0126.SM.5GZXA,GTEX.ZZPU.1326.SM.5GZWS
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
OR4F5,0.04233,0.048,0.147,0.08739,0.07495,0.08668,0.2024,0.1403,0.1016,0.09944,⋯,0,0,0,0.05059,0,0,0.0596,0.1951,0.2498,0.05729


[1] 18446

[1] 4421

In [9]:
# To read the ids of all 4421 samples that we have collected in the previous step

meta = read.table("GTEx_Analysis_v8_RNAseq_samples.ASSIGN.txt", header=F, sep="\t", stringsAsFactors=F)
dim(meta)

[1] 4421    4

In [125]:
# To rename the columns of a in such a way that it reflects the name of the tissue
# To transform "-" into "." in order to make it compatible with the column names in the main data frame

meta.df = meta
meta.df$numerical=rownames(meta)
meta.df$id = paste(meta.df$V4, meta.df$numerical, sep="_")
meta.df$id_sample = (str_replace_all(meta.df$V1, "-", ".")) 
head(meta.df,2)

,V1,V2,V3,V4,numerical,id,id_sample
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,GTEX-1117F-0726-SM-5GIEN,GTEX-1117F,True,Heart_Atrial_Appendage,1,Heart_Atrial_Appendage_1,GTEX.1117F.0726.SM.5GIEN
2,GTEX-111FC-0626-SM-5N9CU,GTEX-111FC,True,Heart_Atrial_Appendage,2,Heart_Atrial_Appendage_2,GTEX.111FC.0626.SM.5N9CU


In [12]:
# to create a  backup data frame for a
b = a 
dim(b)

[1] 18446  4421

In [24]:
# Shall we need to keep the data of each tissue in a separate data frame,
# in order to apply a more sophisticated gene selection technique
# As we do work with only 10 tissues, we separate the data one by one

In [13]:
Heart = read.table("GTEx_Analysis_v8_RNAseq_samples.Heart.txt",  header=F, sep="\t", stringsAsFactors=F)
Heart$numerical = rownames(Heart)
Heart$id = paste(Heart$V4, Heart$numerical, sep="_")
Heart$id_sample = (str_replace_all(Heart$V1, "-", ".")) 
head(Heart, n=1)
dim(Heart)



Kidney = read.table("GTEx_Analysis_v8_RNAseq_samples.Kidney.txt",  header=F, sep="\t", stringsAsFactors=F)
Kidney$numerical = rownames(Kidney)
Kidney$id = paste(Kidney$V4, Kidney$numerical, sep="_")
Kidney$id_sample = (str_replace_all(Kidney$V1, "-", ".")) 
head(Kidney, n=1)
dim(Kidney)



Liver = read.table("GTEx_Analysis_v8_RNAseq_samples.Liver.txt",  header=F, sep="\t", stringsAsFactors=F)
Liver$numerical = rownames(Liver)
Liver$id = paste(Liver$V4, Liver$numerical, sep="_")
Liver$id_sample = (str_replace_all(Liver$V1, "-", ".")) 
head(Liver, n=1)
dim(Liver)



Lung = read.table("GTEx_Analysis_v8_RNAseq_samples.Lung.txt",  header=F, sep="\t", stringsAsFactors=F)
Lung$numerical = rownames(Lung)
Lung$id = paste(Lung$V4, Lung$numerical, sep="_")
Lung$id_sample = (str_replace_all(Lung$V1, "-", ".")) 
head(Lung, n=1)
dim(Lung)



Muscle = read.table("GTEx_Analysis_v8_RNAseq_samples.Muscle.txt",  header=F, sep="\t", stringsAsFactors=F)
Muscle$numerical = rownames(Muscle)
Muscle$id = paste(Muscle$V4, Muscle$numerical, sep="_")
Muscle$id_sample = (str_replace_all(Muscle$V1, "-", ".")) 
head(Muscle, n=1)
dim(Muscle)


Pancreas = read.table("GTEx_Analysis_v8_RNAseq_samples.Pancreas.txt",  header=F, sep="\t", stringsAsFactors=F)
Pancreas$numerical = rownames(Pancreas)
Pancreas$id = paste(Pancreas$V4, Pancreas$numerical, sep="_")
Pancreas$id_sample = (str_replace_all(Pancreas$V1, "-", ".")) 
head(Pancreas, n=1)
dim(Pancreas)


Pituitary = read.table("GTEx_Analysis_v8_RNAseq_samples.Pituitary.txt",  header=F, sep="\t", stringsAsFactors=F)
Pituitary$numerical = rownames(Pituitary)
Pituitary$id = paste(Pituitary$V4, Pituitary$numerical, sep="_")
Pituitary$id_sample = (str_replace_all(Pituitary$V1, "-", ".")) 
head(Pituitary, n=1)
dim(Pituitary)



Spleen = read.table("GTEx_Analysis_v8_RNAseq_samples.Spleen.txt",  header=F, sep="\t", stringsAsFactors=F)
Spleen$numerical = rownames(Spleen)
Spleen$id = paste(Spleen$V4, Spleen$numerical, sep="_")
Spleen$id_sample = (str_replace_all(Spleen$V1, "-", ".")) 
head(Spleen, n=1)
dim(Spleen)



Stomach = read.table("GTEx_Analysis_v8_RNAseq_samples.Stomach.txt",  header=F, sep="\t", stringsAsFactors=F)
Stomach$numerical = rownames(Stomach)
Stomach$id = paste(Stomach$V4, Stomach$numerical, sep="_")
Stomach$id_sample = (str_replace_all(Stomach$V1, "-", ".")) 
head(Stomach, n=1)
dim(Stomach)



Thyroid = read.table("GTEx_Analysis_v8_RNAseq_samples.Thyroid.txt",  header=F, sep="\t", stringsAsFactors=F)
Thyroid$numerical = rownames(Thyroid)
Thyroid$id = paste(Thyroid$V4, Thyroid$numerical, sep="_")
Thyroid$id_sample = (str_replace_all(Thyroid$V1, "-", ".")) 
head(Thyroid, n=1)
dim(Thyroid)


,V1,V2,V3,V4,numerical,id,id_sample
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,GTEX-1117F-0726-SM-5GIEN,GTEX-1117F,True,Heart_Atrial_Appendage,1,Heart_Atrial_Appendage_1,GTEX.1117F.0726.SM.5GIEN


[1] 861   7

,V1,V2,V3,V4,numerical,id,id_sample
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,GTEX-11GS4-2326-SM-5A5KS,GTEX-11GS4,True,Kidney_Cortex,1,Kidney_Cortex_1,GTEX.11GS4.2326.SM.5A5KS


[1] 89  7

,V1,V2,V3,V4,numerical,id,id_sample
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,GTEX-11DXY-0526-SM-5EGGQ,GTEX-11DXY,False,Liver,1,Liver_1,GTEX.11DXY.0526.SM.5EGGQ


...

[1] 241   7

,V1,V2,V3,V4,numerical,id,id_sample
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,GTEX-111CU-0926-SM-5EGIK,GTEX-111CU,True,Stomach,1,Stomach_1,GTEX.111CU.0926.SM.5EGIK


[1] 359   7

,V1,V2,V3,V4,numerical,id,id_sample
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,GTEX-111CU-0226-SM-5GZXC,GTEX-111CU,True,Thyroid,1,Thyroid_1,GTEX.111CU.0226.SM.5GZXC


[1] 653   7

In [15]:
# To separate the data frame per tissue

In [166]:
Heart.df = b %>% select(any_of(Heart$id_sample))
dim(Heart.df)
head(Heart.df,1)
write.table(Heart.df, file = paste(name, "Heart", "txt", sep="."), 
            append = FALSE, quote = FALSE, 
            sep = "\t", row.names = TRUE, col.names = TRUE)


Kidney.df = b %>% select(any_of(Kidney$id_sample))
dim(Kidney.df)
head(Kidney.df,1)
write.table(Kidney.df, file = paste(name, "Kidney", "txt", sep="."), 
            append = FALSE, quote = FALSE, 
            sep = "\t", row.names = TRUE, col.names = TRUE)

Liver.df = b %>% select(any_of(Liver$id_sample))
dim(Liver.df)
head(Liver.df,1)
write.table(Liver.df, file = paste(name, "Liver", "txt", sep="."), 
            append = FALSE, quote = FALSE, 
            sep = "\t", row.names = TRUE, col.names = TRUE)

Lung.df = b %>% select(any_of(Lung$id_sample))
dim(Lung.df)
head(Lung.df,1)
write.table(Lung.df, file = paste(name, "Lung", "txt", sep="."), 
            append = FALSE, quote = FALSE, 
            sep = "\t", row.names = TRUE, col.names = TRUE)

Muscle.df = b %>% select(any_of(Muscle$id_sample))
dim(Muscle.df)
head(Muscle.df,1)
write.table(Muscle.df, file = paste(name, "Muscle", "txt", sep="."), 
            append = FALSE, quote = FALSE, 
            sep = "\t", row.names = TRUE, col.names = TRUE)

Pancreas.df = b %>% select(any_of(Pancreas$id_sample))
dim(Pancreas.df)
head(Pancreas.df,1)
write.table(Pancreas.df, file = paste(name, "Pancreas", "txt", sep="."), 
            append = FALSE, quote = FALSE, 
            sep = "\t", row.names = TRUE, col.names = TRUE)

Pituitary.df = b %>% select(any_of(Pituitary$id_sample))
dim(Pituitary.df)
head(Pituitary.df,1)
write.table(Pituitary.df, file = paste(name, "Pituitary", "txt", sep="."), 
            append = FALSE, quote = FALSE, 
            sep = "\t", row.names = TRUE, col.names = TRUE)

Spleen.df = b %>% select(any_of(Spleen$id_sample))
dim(Spleen.df)
head(Spleen.df,1)
write.table(Spleen.df, file = paste(name, "Spleen", "txt", sep="."), 
            append = FALSE, quote = FALSE, 
            sep = "\t", row.names = TRUE, col.names = TRUE)

Stomach.df = b %>% select(any_of(Stomach$id_sample))
dim(Stomach.df)
head(Stomach.df,1)
write.table(Stomach.df, file = paste(name, "Stomach", "txt", sep="."), 
            append = FALSE, quote = FALSE, 
            sep = "\t", row.names = TRUE, col.names = TRUE)

Thyroid.df = b %>% select(any_of(Thyroid$id_sample))
dim(Thyroid.df)
head(Thyroid.df,1)
write.table(Thyroid.df, file = paste(name, "Thyroid", "txt", sep="."), 
            append = FALSE, quote = FALSE, 
            sep = "\t", row.names = TRUE, col.names = TRUE)

[1] 18446   861

,GTEX.1117F.0726.SM.5GIEN,GTEX.111FC.0626.SM.5N9CU,GTEX.111FC.0826.SM.5GZWO,GTEX.111VG.0326.SM.5GZX7,GTEX.111YS.0326.SM.5GZZ3,GTEX.111YS.0426.SM.5987O,GTEX.1122O.0626.SM.5N9B9,GTEX.1122O.0826.SM.5GICV,GTEX.117XS.0726.SM.5H131,GTEX.117YW.0326.SM.5N9CY,⋯,GTEX.ZYFD.2026.SM.5E459,GTEX.ZYFG.0426.SM.5E43M,GTEX.ZYFG.0526.SM.5GZXX,GTEX.ZYT6.0926.SM.5GIEM,GTEX.ZYT6.1726.SM.5E44P,GTEX.ZYVF.1826.SM.5E44F,GTEX.ZYW4.0926.SM.59HJS,GTEX.ZZPT.0926.SM.5GICZ,GTEX.ZZPU.0926.SM.5GZYT,GTEX.ZZPU.1126.SM.5N9CW
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
OR4F5,0.04233,0.048,0.14700,0.08739,0.07495,0.08668,0.20240,0.1403,0.10160,0.09944,⋯,0.11300,0.1398,0.00000,0.03603,0.11200,0.18270,0.2386,0.06841,0.06997,0.12900
OR4F29,0.00000,0.000,0.11490,0.05696,0.03664,0.08474,0.00000,0.0343,0.06622,0.09721,⋯,0.03684,0.1366,0.09622,0.03522,0.03651,0.17870,0.1696,0.00000,0.10260,0.00000
OR4F16,0.04138,0.000,0.02873,0.11390,0.03664,0.02825,0.06595,0.0343,0.09933,0.04861,⋯,0.22100,0.1093,0.16040,0.03522,0.05476,0.07657,0.2332,0.10030,0.03420,0.03154
SAMD11,11.39000,18.940,1.59900,10.90000,0.95480,0.29140,1.66500,0.3259,9.38300,0.73890,⋯,4.40000,0.3487,2.57700,9.15000,0.35680,9.84600,0.3972,4.55700,0.64060,6.65200
NOC2L,26.52000,40.990,21.96000,16.94000,28.45000,21.76000,34.88000,31.1300,20.39000,9.71600,⋯,52.84000,25.9000,21.84000,31.91000,4.64800,30.88000,19.6200,29.06000,28.40000,34.88000
KLHL17,6.50300,7.844,3.57900,4.99000,2.42400,2.47000,3.91600,2.7560,4.02800,2.18800,⋯,3.09100,2.5840,2.28700,5.99000,1.73200,4.36500,2.2300,9.33400,3.64200,6.38100


[1] 18446    89

,GTEX.11GS4.2326.SM.5A5KS,GTEX.11OF3.1326.SM.5N9FJ,GTEX.11TTK.1926.SM.5PNW8,GTEX.12696.0926.SM.5FQTV,GTEX.12WSG.0826.SM.5EQ5A,GTEX.13112.2126.SM.5GCO4,GTEX.1399S.0526.SM.5IJG8,GTEX.13NYB.1726.SM.5N9G2,GTEX.13O1R.2526.SM.5N9FW,GTEX.13OVI.1126.SM.5KLZF,⋯,GTEX.XPVG.0526.SM.4B65N,GTEX.Y5V5.1226.SM.EYYVP,GTEX.Y5V6.2026.SM.5IFHO,GTEX.ZC5H.1726.SM.5HL7X,GTEX.ZDXO.0226.SM.4WKH7,GTEX.ZE9C.1426.SM.4WKGM,GTEX.ZLFU.0926.SM.5P9F8,GTEX.ZVZP.0926.SM.5GIDB,GTEX.ZYFG.1626.SM.5GZYY,GTEX.ZYT6.2226.SM.5GIC9
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
OR4F5,0.03939,0.13430,0.00,0.00000,0.00000,0.08239,0.33860,0.08503,0.13610,0.15660,⋯,0.08677,0.04811,0.02756,0.1103,0.08474,0.00000,0.11670,0.09065,0.1046,0.1424
OR4F29,0.11550,0.13130,0.00,0.00000,0.00000,0.04027,0.03009,0.02771,0.11090,0.07657,⋯,0.02828,0.09407,0.08084,0.1079,0.08285,0.05707,0.08557,0.08862,0.1363,0.0000
OR4F16,0.03851,0.03753,0.00,0.07983,0.04106,0.20140,0.00000,0.02771,0.02218,0.03828,⋯,0.16970,0.02352,0.05389,0.0000,0.08285,0.11410,0.05705,0.13290,0.1363,0.0000
SAMD11,11.66000,4.95600,21.96,12.54000,7.29000,19.90000,19.41000,7.53700,5.74400,28.29000,⋯,9.83300,7.08000,4.29400,6.4120,10.07000,36.54000,25.07000,16.02000,12.4400,41.4600
NOC2L,24.56000,13.80000,59.97,54.30000,42.01000,43.57000,58.05000,34.80000,33.59000,52.63000,⋯,53.45000,21.10000,26.47000,21.4300,15.11000,87.25000,54.19000,50.14000,53.9600,41.4400
KLHL17,7.91000,3.94100,16.26,16.60000,13.38000,18.31000,20.02000,5.64200,10.34000,16.71000,⋯,12.16000,6.66300,5.11300,13.1600,6.76300,28.01000,16.27000,15.67000,9.4670,17.4400


[1] 18446   226

...

,GTEX.111CU.0426.SM.5GZY1,GTEX.111FC.1326.SM.5N9D9,GTEX.1122O.0526.SM.5N9DM,GTEX.117YX.2126.SM.5GIEL,GTEX.11DXX.0726.SM.5H12X,GTEX.11EQ9.0426.SM.5A5JY,GTEX.11LCK.0126.SM.5A5M5,GTEX.11NSD.0426.SM.5N9CR,GTEX.11P7K.0426.SM.5BC5H,GTEX.11P81.1326.SM.5GU5S,⋯,GTEX.ZP4G.0326.SM.4YCEF,GTEX.ZPCL.0626.SM.DNZZ6,GTEX.ZPIC.0126.SM.DNZZ9,GTEX.ZPU1.1026.SM.4YCEQ,GTEX.ZTTD.1726.SM.57WEL,GTEX.ZV6S.0826.SM.5NQ6Z,GTEX.ZVP2.0526.SM.51MSC,GTEX.ZVZP.0726.SM.59HKA,GTEX.ZYFG.0726.SM.5GIDX,GTEX.ZZPU.0126.SM.5E446
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
OR4F5,0.03451,0.00,0.00,0.04146,0.05517,0.00000,0.03895,0.00,0.0000,0.10870,⋯,0.1184,0.04096,0.03638,0.05738,0.11370,0.00,0.08744,0.05096,0.05353,0.10650
OR4F29,0.03374,0.00,0.00,0.04053,0.00000,0.04028,0.00000,0.00,0.1065,0.05311,⋯,0.0000,0.00000,0.03556,0.11220,0.00000,0.00,0.00000,0.00000,0.10470,0.06939
OR4F16,0.03374,0.00,0.00,0.00000,0.05394,0.00000,0.03808,0.00,0.1598,0.05311,⋯,0.0000,0.00000,0.07113,0.11220,0.05559,0.00,0.00000,0.00000,0.00000,0.06939
SAMD11,21.00000,123.90,14.30,11.30000,8.20000,15.27000,13.13000,30.91,18.9700,32.48000,⋯,23.1500,17.83000,31.02000,16.86000,4.96500,23.29,21.14000,21.92000,16.71000,17.88000
NOC2L,64.49000,63.42,64.59,47.02000,59.56000,50.35000,55.57000,59.98,48.7800,55.42000,⋯,70.7500,47.62000,73.91000,67.16000,90.21000,47.23,55.45000,49.70000,47.85000,44.21000
KLHL17,18.05000,38.04,20.01,19.37000,23.36000,16.47000,21.59000,27.05,15.8800,20.79000,⋯,17.5400,20.53000,24.52000,18.30000,36.29000,20.80,23.79000,21.20000,15.96000,14.15000


[1] 18446   359

,GTEX.111CU.0926.SM.5EGIK,GTEX.111YS.1126.SM.5GZYQ,GTEX.1122O.1926.SM.5EGIQ,GTEX.117YW.2226.SM.5N9DB,GTEX.117YX.1026.SM.5H11V,GTEX.11DXX.1326.SM.5GIDZ,GTEX.11EI6.2426.SM.5PNVS,GTEX.11EM3.1026.SM.5A5KL,GTEX.11EQ9.1226.SM.5987E,GTEX.11GSP.2126.SM.5HL5E,⋯,GTEX.ZVT2.1626.SM.51MRC,GTEX.ZVT3.2126.SM.59HL2,GTEX.ZVZP.1726.SM.5GZWY,GTEX.ZXES.1426.SM.5NQ8S,GTEX.ZY6K.0726.SM.5A5L8,GTEX.ZYFG.1326.SM.5GICJ,GTEX.ZYVF.2726.SM.5GID4,GTEX.ZYY3.1726.SM.5EGH3,GTEX.ZZ64.0426.SM.5E43F,GTEX.ZZPU.1426.SM.5GZZ6
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
OR4F5,0.1335,0.1508,0.1524,0.04992,0.19910,0.04578,0.04205,0.05578,0.04767,0.0000,⋯,0.1604,0.08514,0.08101,0.07967,0.00000,0.0000,0.0000,0.13690,0.1666,0.12690
OR4F29,0.0000,0.0983,0.1986,0.04880,0.03892,0.04476,0.00000,0.05454,0.04660,0.0581,⋯,0.0000,0.00000,0.03960,0.11680,0.00000,0.1485,0.0000,0.04462,0.1628,0.00000
OR4F16,0.0000,0.2458,0.2483,0.00000,0.03892,0.04476,0.04111,0.00000,0.09320,0.0000,⋯,0.1176,0.12490,0.11880,0.00000,0.04662,0.1485,0.1096,0.17850,0.1221,0.04136
SAMD11,2.0560,1.3080,0.6740,1.70900,3.95200,2.35700,3.32500,0.45890,6.00900,1.9400,⋯,1.7560,3.42300,2.40800,2.66400,3.63200,1.8670,25.4500,1.74400,1.3810,1.76300
NOC2L,52.8700,32.5700,35.2000,55.00000,43.14000,41.76000,25.13000,40.86000,62.56000,32.2400,⋯,32.1000,27.31000,32.69000,35.34000,43.56000,32.5800,84.2600,24.31000,35.1500,41.23000
KLHL17,12.4300,8.9450,8.2840,6.48800,11.79000,14.56000,12.69000,8.74600,7.92500,6.8990,⋯,9.5540,11.63000,7.57700,10.23000,12.22000,9.7090,24.2800,11.46000,8.2750,9.94300


[1] 18446   653

,GTEX.111CU.0226.SM.5GZXC,GTEX.111FC.1026.SM.5GZX1,GTEX.111VG.0526.SM.5N9BW,GTEX.111YS.0726.SM.5GZY8,GTEX.1122O.0226.SM.5N9DA,GTEX.1128S.0126.SM.5H12S,GTEX.113JC.0126.SM.5EGJW,GTEX.117XS.0526.SM.5987Q,GTEX.117YW.0126.SM.5EGGN,GTEX.117YX.1226.SM.5H11S,⋯,GTEX.ZY6K.0226.SM.5SIAY,GTEX.ZYFC.0926.SM.5GZWW,GTEX.ZYFD.0826.SM.5NQ9A,GTEX.ZYFG.0626.SM.5GZYA,GTEX.ZYT6.0426.SM.5GID3,GTEX.ZYVF.1126.SM.5E458,GTEX.ZYW4.1126.SM.5SI99,GTEX.ZYY3.1926.SM.5GZXS,GTEX.ZZ64.0126.SM.5GZXA,GTEX.ZZPU.1326.SM.5GZWS
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
OR4F5,0.04395,0.05304,0.04320,0.0000,0.04593,0.06872,0.000,0.07647,0.00000,0.1066,⋯,0.00000,0.00000,0.000,0.05059,0.00000,0.0000,0.0596,0.19510,0.2498,0.05729
OR4F29,0.08594,0.05186,0.04224,0.0000,0.08981,0.00000,0.000,0.00000,0.00000,0.1043,⋯,0.05389,0.00000,0.000,0.04945,0.06155,0.2103,0.0000,0.00000,0.0000,0.11200
OR4F16,0.00000,0.00000,0.04224,0.1602,0.00000,0.13440,0.000,0.14950,0.07848,0.1043,⋯,0.05389,0.06385,0.000,0.04945,0.06155,0.0000,0.0000,0.09535,0.0000,0.00000
SAMD11,4.23400,9.95300,11.50000,7.9850,3.09600,2.60800,3.813,6.27100,3.11000,1.8680,⋯,4.27200,15.22000,5.599,4.75300,9.90800,3.7300,0.5220,8.34800,9.0150,1.24700
NOC2L,75.24000,62.23000,36.38000,63.9700,49.26000,90.78000,66.790,73.13000,66.30000,57.1500,⋯,65.57000,70.77000,103.900,57.76000,71.05000,96.0900,69.3800,49.24000,60.4200,60.42000
KLHL17,9.02600,13.35000,17.09000,13.3500,13.11000,16.87000,10.080,17.92000,11.42000,14.2500,⋯,13.88000,7.54000,9.333,12.58000,18.61000,23.4600,10.9000,19.00000,11.2400,20.48000


In [126]:
# to separate the META DATA FRAME into LIVER versus ANY OTHER TISSUE
meta.df$class = ifelse(meta.df$V4 == "Liver", "Liver", "Other")
head(meta.df, 1)

meta.df.Liver = subset(meta.df, V4 == "Liver")
head(meta.df.Liver, 1)
dim(meta.df.Liver)

meta.df.Other = subset(meta.df, V4 != "Liver")
head(meta.df.Other, 1)
dim(meta.df.Other)


,V1,V2,V3,V4,numerical,id,id_sample,class
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,GTEX-1117F-0726-SM-5GIEN,GTEX-1117F,True,Heart_Atrial_Appendage,1,Heart_Atrial_Appendage_1,GTEX.1117F.0726.SM.5GIEN,Other


,V1,V2,V3,V4,numerical,id,id_sample,class
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
951,GTEX-11DXY-0526-SM-5EGGQ,GTEX-11DXY,False,Liver,951,Liver_951,GTEX.11DXY.0526.SM.5EGGQ,Liver


[1] 226   8

,V1,V2,V3,V4,numerical,id,id_sample,class
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,GTEX-1117F-0726-SM-5GIEN,GTEX-1117F,True,Heart_Atrial_Appendage,1,Heart_Atrial_Appendage_1,GTEX.1117F.0726.SM.5GIEN,Other


[1] 4195    8

In [127]:
Liver.only.df = b %>% select(any_of(meta.df.Liver$id_sample))
head(Liver.only.df,1)
dim(Liver.only.df)
write.table(Liver.only.df, file = paste(name, "only.Liver", "txt", sep="."), 
            append = FALSE, quote = FALSE, 
            sep = "\t", row.names = TRUE, col.names = TRUE)


Other.only.df = b %>% select(any_of(meta.df.Other$id_sample))
head(Other.only.df,1)
dim(Other.only.df)

write.table(Other.only.df, file = paste(name, "only.Other", "txt", sep="."), 
            append = FALSE, quote = FALSE, 
            sep = "\t", row.names = TRUE, col.names = TRUE)

,GTEX.11DXY.0526.SM.5EGGQ,GTEX.11DXZ.0126.SM.5EGGY,GTEX.11EQ9.0526.SM.5A5JZ,GTEX.11GSP.0626.SM.5986T,GTEX.11NUK.1226.SM.5P9GM,GTEX.11NV4.1326.SM.5HL6V,GTEX.11OF3.0726.SM.5BC4Z,GTEX.11TT1.1726.SM.5EQLJ,GTEX.11TUW.1726.SM.5BC5C,GTEX.11WQC.0726.SM.5EQMR,⋯,GTEX.ZF29.2026.SM.DNZYW,GTEX.ZF2S.3026.SM.4WWCH,GTEX.ZPU1.0826.SM.57WG2,GTEX.ZTPG.1426.SM.51MT3,GTEX.ZVP2.0626.SM.51MSO,GTEX.ZVT3.1626.SM.5GU66,GTEX.ZVT4.0626.SM.5E45T,GTEX.ZYT6.0626.SM.5E45V,GTEX.ZYY3.0626.SM.5NQ6W,GTEX.ZZPU.0426.SM.5GZYH
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
OR4F5,0,0.09123,0.03197,0.1352,0.1199,0.05194,0.1338,0.1551,0.06967,0.07092,⋯,0.02642,0,0.06901,0.1233,0.07165,0.08659,0.06814,0.124,0,0.08212


[1] 18446   226

,GTEX.1117F.0726.SM.5GIEN,GTEX.111FC.0626.SM.5N9CU,GTEX.111FC.0826.SM.5GZWO,GTEX.111VG.0326.SM.5GZX7,GTEX.111YS.0326.SM.5GZZ3,GTEX.111YS.0426.SM.5987O,GTEX.1122O.0626.SM.5N9B9,GTEX.1122O.0826.SM.5GICV,GTEX.117XS.0726.SM.5H131,GTEX.117YW.0326.SM.5N9CY,⋯,GTEX.ZY6K.0226.SM.5SIAY,GTEX.ZYFC.0926.SM.5GZWW,GTEX.ZYFD.0826.SM.5NQ9A,GTEX.ZYFG.0626.SM.5GZYA,GTEX.ZYT6.0426.SM.5GID3,GTEX.ZYVF.1126.SM.5E458,GTEX.ZYW4.1126.SM.5SI99,GTEX.ZYY3.1926.SM.5GZXS,GTEX.ZZ64.0126.SM.5GZXA,GTEX.ZZPU.1326.SM.5GZWS
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
OR4F5,0.04233,0.048,0.147,0.08739,0.07495,0.08668,0.2024,0.1403,0.1016,0.09944,⋯,0,0,0,0.05059,0,0,0.0596,0.1951,0.2498,0.05729


[1] 18446  4195

In [ ]:
# To compute the SPECIFIC GENES based on the MEDIAN of GENE EPRESSION in LIVER versus OTHER tissues

In [22]:
head(Liver.only.df,2)
head(Other.only.df,2)

,GTEX.11DXY.0526.SM.5EGGQ,GTEX.11DXZ.0126.SM.5EGGY,GTEX.11EQ9.0526.SM.5A5JZ,GTEX.11GSP.0626.SM.5986T,GTEX.11NUK.1226.SM.5P9GM,GTEX.11NV4.1326.SM.5HL6V,GTEX.11OF3.0726.SM.5BC4Z,GTEX.11TT1.1726.SM.5EQLJ,GTEX.11TUW.1726.SM.5BC5C,GTEX.11WQC.0726.SM.5EQMR,⋯,GTEX.ZF29.2026.SM.DNZYW,GTEX.ZF2S.3026.SM.4WWCH,GTEX.ZPU1.0826.SM.57WG2,GTEX.ZTPG.1426.SM.51MT3,GTEX.ZVP2.0626.SM.51MSO,GTEX.ZVT3.1626.SM.5GU66,GTEX.ZVT4.0626.SM.5E45T,GTEX.ZYT6.0626.SM.5E45V,GTEX.ZYY3.0626.SM.5NQ6W,GTEX.ZZPU.0426.SM.5GZYH
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
OR4F5,0,0.09123,0.03197,0.13520,0.11990,0.05194,0.13380,0.1551,0.06967,0.07092,⋯,0.02642,0.00000,0.06901,0.12330,0.07165,0.08659,0.06814,0.12400,0,0.08212
OR4F29,0,0.13380,0.03126,0.03305,0.09376,0.02539,0.08722,0.1011,0.06811,0.10400,⋯,0.02583,0.03943,0.06747,0.08037,0.10510,0.04233,0.09993,0.09092,0,0.08028


,GTEX.1117F.0726.SM.5GIEN,GTEX.111FC.0626.SM.5N9CU,GTEX.111FC.0826.SM.5GZWO,GTEX.111VG.0326.SM.5GZX7,GTEX.111YS.0326.SM.5GZZ3,GTEX.111YS.0426.SM.5987O,GTEX.1122O.0626.SM.5N9B9,GTEX.1122O.0826.SM.5GICV,GTEX.117XS.0726.SM.5H131,GTEX.117YW.0326.SM.5N9CY,⋯,GTEX.ZY6K.0226.SM.5SIAY,GTEX.ZYFC.0926.SM.5GZWW,GTEX.ZYFD.0826.SM.5NQ9A,GTEX.ZYFG.0626.SM.5GZYA,GTEX.ZYT6.0426.SM.5GID3,GTEX.ZYVF.1126.SM.5E458,GTEX.ZYW4.1126.SM.5SI99,GTEX.ZYY3.1926.SM.5GZXS,GTEX.ZZ64.0126.SM.5GZXA,GTEX.ZZPU.1326.SM.5GZWS
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
OR4F5,0.04233,0.048,0.1470,0.08739,0.07495,0.08668,0.2024,0.1403,0.10160,0.09944,⋯,0.00000,0,0,0.05059,0.00000,0.0000,0.0596,0.1951,0.2498,0.05729
OR4F29,0.00000,0.000,0.1149,0.05696,0.03664,0.08474,0.0000,0.0343,0.06622,0.09721,⋯,0.05389,0,0,0.04945,0.06155,0.2103,0.0000,0.0000,0.0000,0.11200


In [128]:
# To COMPUTE the MEDIAN VALUE of GENE EXPRESSION in these two DATA FRAMES 
# LIVER expression data frame
# OTHER tissue expression data frame

Liver.only.df$median = apply(Liver.only.df, 1, median)

# to create a new data frame that contains only the MEDIAN VALUES
Liver.only.median = data.frame()
Liver.only.median = subset(Liver.only.df, select="median")
Liver.only.median$gene = rownames(Liver.only.df)
Liver.only.median$median_tpm_Liver = Liver.only.df$median
head(Liver.only.median, 2)

,median,gene,median_tpm_Liver
,<dbl>,<chr>,<dbl>
OR4F5,0.060540,OR4F5,0.060540
OR4F29,0.039795,OR4F29,0.039795


In [129]:
# To COMPUTE the MEDIAN VALUE of GENE EXPRESSION in these two DATA FRAMES 
Other.only.df$median = apply(Other.only.df, 1, median)

# to create a new data frame that contains only the MEDIAN VALUES
Other.only.median = data.frame()
Other.only.median = subset(Other.only.df, select="median")
Other.only.median$gene = rownames(Other.only.df)
Other.only.median$median_tpm_Other = Other.only.df$median
head(Other.only.median, 2)

,median,gene,median_tpm_Other
,<dbl>,<chr>,<dbl>
OR4F5,0.05140,OR4F5,0.05140
OR4F29,0.03968,OR4F29,0.03968


In [ ]:
# to combine the two dataframes

In [130]:
# We combine the two dataframes into a simpler data frame 
# Liver.only.median
# Other.only.median

median.df = cbind(Liver.only.median, Other.only.median)
head(median.df,2)
median.df.simple = subset(median.df, select = c(median_tpm_Liver, median_tpm_Other))
median.df.simple$LO = median.df.simple$median_tpm_Liver / median.df.simple$median_tpm_Other
median.df.simple$gene  = rownames(median.df.simple) 
head(median.df.simple, 2)

,median,gene,median_tpm_Liver,median,gene,median_tpm_Other
,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>
OR4F5,0.060540,OR4F5,0.060540,0.05140,OR4F5,0.05140
OR4F29,0.039795,OR4F29,0.039795,0.03968,OR4F29,0.03968


,median_tpm_Liver,median_tpm_Other,LO,gene
,<dbl>,<dbl>,<dbl>,<chr>
OR4F5,0.060540,0.05140,1.177821,OR4F5
OR4F29,0.039795,0.03968,1.002898,OR4F29


In [131]:
summary(median.df$median_tpm_Liver)
summary(median.df$median_tpm_Other)

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
    0.00     0.15     2.54    46.19    10.54 58775.00 

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
    0.00     0.30     4.55    34.32    15.76 36350.00 

In [132]:
# to exclude the genes that may have noisy expression in the other tissues (median_tpm > 0.2)
# and to keep the genes that are expressed in the liver (TPM > 1) and liver-specific ( LO_FC > 1.2 )


median.df.liver.specific = subset(median.df.simple, (median_tpm_Liver > 1) & (median_tpm_Other > 0.2) & 
                                                    (LO > 1.2))

dim(median.df.liver.specific)
head(median.df.liver.specific, 2)



[1] 1938    4

,median_tpm_Liver,median_tpm_Other,LO,gene
,<dbl>,<dbl>,<dbl>,<chr>
ISG15,28.935,20.460,1.414223,ISG15
SCNN1D,5.877,4.422,1.329037,SCNN1D


In [133]:
# to write the set of liver specific genes in a file for further use

write.table(median.df.liver.specific, file = paste(name, "liver.specific.genes", "txt", sep="."), 
            append = FALSE, quote = FALSE, 
            sep = "\t", row.names = FALSE, col.names = TRUE)